[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tduckenfield/wisa_motionmag_tutorial/blob/main/start_here.ipynb)

<figure>
<center>
<img src="https://github.com/Tduckenfield/wisa_motionmag_tutorial/blob/main/figs/Anfinogentov_mm_example.jpg?raw=true">
<figcaption>[Figure 7 of Anfinogentov et al 2022](https://link.springer.com/article/10.1007/s11214-021-00869-w)</figcaption></center>
</figure>

# Motion magnification
Motion magnification is a processing technique which amplifies small transverse quasi-periodic motions of contrasted features in image sequences: it acts as a **microscope for videos**.

Consider the humble microscope: through amplifying the image of small objects, microscopes have allowed us to view the world of tiny things the human eye cannot see. They led to some of the greatest scientific breakthroughs in history - for example cellular biology, forensic science, the existence of bacteria + viruses etc. Can we do the same thing but to small spatial scales changing over time in a video?

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML # To display video
from base64 import b64encode
#from google.colab import drive   # Connect to your google drive if you want to save anything
#drive.mount('/content/gdrive')

!gdown 1l8HjifR4t52D16tx2NGtk0FoxxD3wWDp --output sergey.mp4

# Show video
mp4 = open('/content/sergey.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

This is [Dr Sergey Anfinogentov](https://orcid.org/0000-0002-1107-7420). He has provided everyone with a [code on Github](https://github.com/Sergey-Anfinogentov/motion_magnification) which can motion magnify any video. I aim to help you run motion magnification, and explain how it works!

You can see in the video above, that whilst the video on the left looks quite static, there are a whole range of motions which are much more obvious on the right. **These motions were always there**.

This processing tool has been used by me and others for use in solar physics, notably Sihui Zhong, Dong Li, Sudip Mandal, Yuhang Gao & Valery Nakariakov, all of whom are  here today! By magnifying the transverse motions of coronal structures seen in extreme ultraviolet, we have been able to study in great detail the regime of *decayless kink oscillations*.

These waves are everywhere in the corona, so it is highly likely you could motion magnify any image sequence of the Sun where coronal structures are resolved (e.g. SDO/AIA 17.1 nm) and find some of these transverse waves. To prove me right/wrong, try going through the [mm_solardata](https://github.com/Tduckenfield/wisa_motionmag_tutorial/blob/main/notebooks/mm_solardata.ipynb) notebook and give it a go!   

# Working example
Let us do some motion magnification as a "black box" process. Feel free to use any video! Here I have picked a famous video of a guitar (which is often used in image processing), which you can access from my shared Google Drive - see the readme. 

I **strongly** suggest you take this opportunity and get your own video to motion magnify! Try to find something not too big, and with a relatively stable background. 

In [ ]:
# Access guitar video. Replace with your own!
!gdown 1jK4K3hASDLyV1_IL8i7aHN0Bw_NmPY8Q --output guitar.mp4

# Input video path
mymovie_path = "/content/guitar.mp4"
# For those on the Google drive, something like "/content/gdrive/MyDrive/myvideo.mp4"

# Show video
mp4 = open(mymovie_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

The following cell will convert our video to a numpy array.

In [ ]:
import cv2

videoReader = cv2.VideoCapture(mymovie_path)  # create video object
nFrames = int(videoReader.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(videoReader.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(videoReader.get(cv2.CAP_PROP_FRAME_HEIGHT))

print("Total Number of Frames = ", nFrames)
print("Each frame has dimensions = ",frameHeight,"x", frameWidth)

# Note we specify float32 since necessary for cv2.color_bgr2gray
img_array_rgb = np.zeros((nFrames, frameHeight,frameWidth, 3), np.dtype("float32"))
img_array = np.zeros((nFrames, frameHeight,frameWidth), np.dtype("float32"))

nf = 0
ret = True

while (nf < nFrames  and ret):  # Could do for loop as well
#		print("frame: " + str(nf))
    ret, img_array_rgb[nf] = videoReader.read()
    img_array[nf] = cv2.cvtColor(img_array_rgb[nf], cv2.COLOR_RGB2GRAY)
    nf += 1
#		if ret == False or img is None:
#				print("done")
#				break

#videoWriter = make_video_writer(videoReader, output_video_filename)

videoReader.release()

Take a note in particular of the number of frames. Also, I generally aim for a datacube $\le$ 500 x 1000 x 1000  to get the motion magnification to run without any problems. 

Now we install the necessary library and get the motion magnification code we will use. To learn more about the DTCWT library upon which this MM code is based, see the [understanding_dtcwt](https://github.com/Tduckenfield/wisa_motionmag_tutorial/blob/main/notebooks/understanding_dtwct.ipynb) notebook.

In [ ]:
%pip --quiet install dtcwt
!git clone https://github.com/Sergey-Anfinogentov/motion_magnification.git
%cd /content/motion_magnification
from magnify import *

And now do the magnification! We have to choose two parameters, more on that later. This may take a few minutes depending on the size of the datacube.

In [ ]:
k = 10 # Amplification factor
w = 50 # Width, motions who have a timescale < w will be amplified.

img_mm = magnify_motions_2d(img_array, k, 20)

Now we save out the cube. If you know python, you could directly image the magnified data (as a numpy cube) at this point using `FuncAnimation` etc. I do this later, but in a very crude way.  
For now, let us just admire the magnified video.

In [ ]:
from matplotlib import animation

# Here I choose some standard mp4 parameters. Should adjust for data, e.g. 10 fps on 12s cadence AIA data is too slow!
output_path = '/content/mymovie_mm.mp4'

Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=180)

# Define a rough function to create animation. 
def Create_Animation(image,number_of_files,title):
    ## Create an Array of pictures from a Data Cube
    images=[]
    fig=plt.figure()
    for i in range(number_of_files):
        img_plot=plt.imshow(image[i], cmap='gray')   # Output in greyscale
        images.append([img_plot])
    ani = animation.ArtistAnimation(fig, images, interval=100, blit=True)
    ani.save(title,writer=writer)
    return images

# Create new video.
images=Create_Animation(img_mm,nFrames,output_path)

# Show video
mp4 = open('/content/mymovie_mm.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

And there you have it! Hopefully you can see transverse motions have been amplified, revealing previously imperceptible phenomena. 

Within this github directory, there are some other notebooks focussing on this routine and its use in solar physics in more detail. In light of the themes for this conference - detecting vortex/swirls and identifying MHD wave modes - I suggest you work through [understanding_dtcwt.ipynb](https://colab.research.google.com/github/Tduckenfield/wisa_motionmag_tutorial/blob/main/notebooks/understanding_dtcwt.ipynb). Peruse at your leisure, but I **highly** recommend you give motion magnifying a video of your own a go. It is perfectly possible to take video from your phone and magnify it! Check out [bye.ipynb](https://colab.research.google.com/github/Tduckenfield/wisa_motionmag_tutorial/blob/main/notebooks/bye.ipynb) to see what I mean!